In [ ]:
import pandas as pd
import numpy as np
import random, math, itertools
from scipy.linalg import lu
import matplotlib.pyplot as plt
import matplotlib.font_manager

accs = np.load(open('data/rereads_acc.npy','rb'), allow_pickle=True)

# load in accuracyes from the rereading simulation
p_vals = {2: (accs[0][0], accs[0][5], accs[0][10]),
 4: (accs[2][0], accs[2][5], accs[2][10]),
 8: (accs[6][0], accs[6][5], accs[6][10]),
 16: (accs[14][0], accs[14][5], accs[14][10])}

In [ ]:
def flip_coin(p):
    return random.random() <= p

def create_generating_matrix(n, k):
    I_k = np.eye(k, dtype=int)
    I_nk = np.eye(n - k, dtype=int)

    if k == n:
        return I_k
    
    # find a generating matrix where rank of H is n - k
    while True:
        P = np.random.randint(2, size=(k, n - k))
        H = np.concatenate((P.T, I_nk), axis=1)

        # Check if the rank of H is n - k
        _, _, u = lu(H)
        if np.linalg.matrix_rank(u) == n - k:
            break

    G = np.concatenate((I_k, P), axis=1)
    return G


def encode_message(G, message):
    # Encode the message by multiplying it with the generating matrix G
    encoded_message = np.mod(message @ G, 2)

    return encoded_message

# with probability error_rate, change the message, for each VR
def mutate_message(message, teplet_size, error_rate):
    nbits = int(math.log(teplet_size, 2)) # bits per VR
    for i in range(0, len(message), nbits): 
        if flip_coin(error_rate):
            bit_string = ''.join(str(bit) for bit in message[i:i+nbits])
            current_num = int(bit_string, 2)

            # teplet_size - 2 instead of -1, so that we can exclude selecting current_num
            new_num = random.randint(0, teplet_size-2) 
            if new_num >= current_num:
                # add 1 to ensure new_num != current_num, 
                # i.e. that with probability error_rate, the number actually changes
                new_num += 1 
                
            message[i:i+nbits] = np.array([(new_num >> i) & 1 for i in range(nbits - 1, -1, -1)])
    return message

def find_parity_check_matrix(G, n, k):
    # Get the identity matrix
    I = np.eye(n - k, dtype=int)

    # Extract P from G
    P = G[:, k:]

    # Compute the transpose of P
    P_T = P.transpose()

    # Create the parity-check matrix H
    H = np.concatenate((P_T, I), axis=1)

    return H

def decode_message(H,coset_leaders, encoded_message, n, k):
    decoded_message = syndrome_decoding(H, encoded_message, coset_leaders)

    # Extract the original message by keeping the first k bits
    original_message = decoded_message[:k]

    return original_message

def syndrome_decoding(H, encoded_message, coset_leaders):
    # Compute the syndrome
    decoded_message = encoded_message.copy()
    syndrome = arr_to_num(np.mod(encoded_message @ H.T, 2))
    if syndrome == 0:
        return encoded_message
    
    # coset_leaders = calculate_coset_leaders(H)
    if syndrome not in coset_leaders:
        return decoded_message
    coset = coset_leaders[syndrome][0]
    for idx in coset:
        decoded_message[idx] = int(not encoded_message[idx])
    
    return decoded_message

def arr_to_num(arr):
    return sum([x << (len(arr) - i -1) for i, x in enumerate(arr)])

def calculate_coset_leaders(H):
    es = [H[:, i] for i in range(H.shape[1])]
    syndrome_to_cosets = {}
    # ideally, should calculate for r in range(len(es)), but that can get very very slow.
    # This calculation represents a (slight) underrepresentation of the possible gain in accuracy
    # from error correcting codes
    for r in range(min(6, len(es))):
        for combo in itertools.combinations([i for i in range(len(es))], r):
            if len(combo) < 1:
                continue
            syndrome_arr = np.mod(np.sum([es[c] for c in combo], axis=0), 2)
            syndrome = arr_to_num(syndrome_arr)
            if syndrome in syndrome_to_cosets:
                leading_len =  min([len(s) for s in syndrome_to_cosets[syndrome]])
                if len(combo) == leading_len:
                    syndrome_to_cosets[syndrome].append(combo)
                if len(combo) < leading_len:
                    syndrome_to_cosets[syndrome] = [combo]
            else:
                syndrome_to_cosets[syndrome] = [combo]
    return syndrome_to_cosets

In [ ]:
def calc_data_np(pep_length, rr_idx):
    k_to_ts_to_acc = {}
    # teplet_size refers to alphabet size
    for teplet_size in [2,4,8,16]:
        # iterate over k, number of bits allocated to the message. n-k bits allocated to error correcting codes
        for k in range(1, pep_length*int(math.log(teplet_size,2)) + 1):
            print(k, end = ", ")
            if k not in k_to_ts_to_acc:
                k_to_ts_to_acc[k] = {}

            num_corr = 0
            trials = 50000
            # error rate = 1 - accuracy
            error_rate = 1-p_vals[teplet_size][rr_idx]

            n = pep_length*int(math.log(teplet_size,2))
            G_matrix = create_generating_matrix(n, k)  
            H_matrix = find_parity_check_matrix(G_matrix, n, k) 
            coset_leaders = calculate_coset_leaders(H_matrix)

            # conduct 50k trials for each alphabet size, protein length, and # rereads
            for i in range(trials):              
                # chose a random number to encode, which represents one of the barcodes. 
                # since there are 2^k possible barcodes, we encode a random number in the range [0, 2^k]
                orig_bits = np.array([random.randint(0, 1) for i in range(k)])
                # encode with the generating matrix
                message = encode_message(G_matrix, orig_bits)
                # mutate with probability = error_rate
                mutated_message = mutate_message(message, teplet_size, error_rate)
                # decode
                recovered_bits = decode_message(H_matrix, coset_leaders, mutated_message, n, k)
                assess_recovery = np.equal(recovered_bits, orig_bits).all()
                num_corr += assess_recovery # correct iff exactly equal to original message
                
            k_to_ts_to_acc[k][teplet_size] = num_corr/trials
        print()
    return k_to_ts_to_acc

In [ ]:
colors = {
    5: {0: 'tab:blue', 1: 'tab:green', 2: 'tab:gray'},
    10: {0: 'tab:red', 1: 'tab:orange', 2: 'tab:olive'}
}

def barcode_space(k):
    return 2**k

for protein_size in [10, 5]:
    for reread_idx in [2, 1, 0]:
        # number of rereads (along with alphabet size) determines the error rate 
        rereads = 10 if reread_idx == 2 else 5 if reread_idx == 1 else 1
        # results = np.load(f"./barcode_results/barcode_{rereads}rr_{protein_size}len.npy", allow_pickle=True).item()
        results = calc_data_np(protein_size, reread_idx, results)
        # np.save(f"./barcode_results/barcode_{rereads}rr_{protein_size}len.npy", results)
        for k, dic in results.items():
            max_ts = -1
            max_acc = 0
            for ts, acc in dic.items():
                if acc > max_acc:
                    max_acc, max_ts = acc, ts
            if max_acc > .5:
                plt.scatter(barcode_space(k), max_acc*100, color = colors[protein_size][reread_idx], alpha=.5,
                    s = 80 if max_ts == 8 else 35, marker = 'o' if max_ts == 2 else 'v' if max_ts == 4 else '1' ,
                    linewidths= 2.5 if max_ts == 8  else 1
                    ) 

blue_patch = plt.plot([],[], marker='o',  color='tab:blue', linestyle='none', label='5 VRs,\n0 rereads')
green_patch = plt.plot([],[], marker='o', color='tab:green', linestyle='none', label='5 VRs,\n5 rereads')
gray_patch = plt.plot([],[], marker='o',  color='tab:gray', linestyle='none', label='5 VRs,\n10 rereads')
red_patch = plt.plot([],[], marker='o',  color='tab:red', linestyle='none', label='10 VRs,\n0 rereads')
orange_patch = plt.plot([],[], marker='o', color='tab:orange', linestyle='none', label='10 VRs,\n5 rereads')
olive_patch = plt.plot([],[], marker='o',  color='tab:olive', linestyle='none', label='10 VRs,\n10 rereads')

circ_patch = plt.plot([],[], marker='o',  color='black', linestyle='none', label='2')
tria_patch = plt.plot([],[], marker='v', color='black', linestyle='none', label='4')
star_patch = plt.plot([],[], marker='1',  color='black', linestyle='none', label='8')

plt.xlabel("Barcodes space size")
plt.ylabel("Accuracy (%)")
ax = plt.gca()
ax.set_xscale('log')
plt.legend(title="Alphabet Size")
ax.legend(handles=[blue_patch[0], 
                    green_patch[0],
                    gray_patch[0], 
                    red_patch[0], 
                    orange_patch[0], 
                    olive_patch[0],
                    circ_patch[0], tria_patch[0], star_patch[0]
                    ],
        loc='center left', 
        bbox_to_anchor=(.7, .85)
        )
# plt.title(f"Barcode space vs accuracy")
plt.ylim([50,103])
ax.spines[['right', 'top']].set_visible(False)
ax.spines[['right', 'top']].set_visible(False)
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none',
'font.family':'arial',
}
plt.rcParams.update(new_rc_params)
# plt.savefig(f"barcodes_space.svg", transparent=True , dpi=600, bbox_inches='tight', format='svg')
plt.show()